# Average mutation functional effects for a condition

Import Python modules.
We use `polyclonal` for the plotting:

In [ ]:
import altair as alt

import dms_variants.utils

import pandas as pd

import polyclonal
import polyclonal.plot

This notebook is parameterized by `papermill`.
The next cell is tagged as `parameters` to get the passed parameters.

In [ ]:
# this cell is tagged parameters for `papermill` parameterization
site_numbering_map_csv = None
func_effects_csv = None
latent_effects_csv = None
latent_html = None
functional_html = None
params = None

Read the input data:

In [ ]:
site_numbering_map = pd.read_csv(site_numbering_map_csv).rename(
    columns={"reference_site": "site"}
)

selections = params["selections"]

func_effects = pd.concat(
    [
        pd.read_csv(f"results/func_effects/by_selection/{s}_func_effects.csv").assign(
            selection=s
        )
        for s in selections
    ],
    ignore_index=True,
).assign(times_seen=lambda x: x["times_seen"].astype("Int64"))

## Correlations among selections
Compute the correlations in the mutation effects in terms of functional scores and latent-phenotype effects across selections:

In [ ]:
# We compute for several times seen values, get those:
try:
    init_times_seen = params["plot_kwargs"]["addtl_slider_stats"]["times_seen"]
except KeyError:
    print("No times seen in params, using a value of 3")
    init_times_seen = 3

func_effects_tidy = func_effects.assign(
    mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"]
).melt(
    id_vars=["selection", "mutation", "times_seen"],
    value_vars=["latent_phenotype_effect", "functional_effect"],
    var_name="phenotype",
    value_name="effect",
)

# do analysise for each "times_seen"
func_effects_tidy = pd.concat(
    [
        func_effects_tidy.query("times_seen >= @t").assign(min_times_seen=t)
        for t in [1, init_times_seen, 2 * init_times_seen]
    ]
)


corrs = (
    dms_variants.utils.tidy_to_corr(
        df=func_effects_tidy,
        sample_col="selection",
        label_col="mutation",
        value_col="effect",
        group_cols=["phenotype", "min_times_seen"],
    )
    .assign(r2=lambda x: x["correlation"] ** 2)
    .drop(columns="correlation")
    .assign(
        min_times_seen=lambda x: "min times seen " + x["min_times_seen"].astype(str)
    )
)

for phenotype, phenotype_corr in corrs.groupby("phenotype"):
    corr_chart = (
        alt.Chart(phenotype_corr)
        .encode(
            alt.X("selection_1", title=None),
            alt.Y("selection_2", title=None),
            column=alt.Column("min_times_seen", title=None),
            color=alt.Color("r2", scale=alt.Scale(zero=True)),
            tooltip=[
                alt.Tooltip(c, format=".3g") if c == "r2" else c
                for c in ["phenotype", "selection_1", "selection_2", "r2"]
            ],
        )
        .mark_rect(stroke="black")
        .properties(width=alt.Step(15), height=alt.Step(15), title=phenotype)
        .configure_axis(labelLimit=500)
    )

    display(corr_chart)

## Average functional effects
We average the functional effects across selections.
The resulting file has the average functional effect for each selection (either mean or median), the mean times seen across selections, the number of selections each mutation is seen in, and the effect (times seen) for each individual selection.

In [ ]:
avg_method = params["avg_method"]
assert avg_method in {"mean", "median"}, avg_method

func_effects_by_phenotype = func_effects.melt(
    id_vars=["site", "wildtype", "mutant", "times_seen", "selection"],
    value_vars=["functional_effect", "latent_phenotype_effect"],
    var_name="phenotype",
    value_name="effect",
)

avg_func_effects = (
    func_effects_by_phenotype.groupby(
        ["phenotype", "site", "wildtype", "mutant"], as_index=False
    )
    .aggregate(
        effect=pd.NamedAgg("effect", avg_method),
        times_seen=pd.NamedAgg("times_seen", "sum"),
        n_selections=pd.NamedAgg("site", "count"),
    )
    .assign(
        times_seen=lambda x: (x["times_seen"] / x["n_selections"]).where(
            x["mutant"] != x["wildtype"],
            pd.NA,
        )
    )
)

assert not set(selections).intersection(avg_func_effects.columns)

# add per-selection effects (times_seen)
avg_func_effects = avg_func_effects.merge(
    (
        func_effects_by_phenotype.assign(
            effect_times_seen=lambda x: (
                x["effect"].map(lambda e: f"{e:.2f}")
                + (" (" + x["times_seen"].astype(str) + ")").where(
                    x["mutant"] != x["wildtype"],
                    "",
                )
            )
        )
        .pivot_table(
            index=[
                "site",
                "wildtype",
                "mutant",
                "phenotype",
            ],
            values="effect_times_seen",
            columns="selection",
            aggfunc=lambda s: ",".join(s),
        )
        .reset_index()
    ),
    on=["phenotype", "site", "wildtype", "mutant"],
    validate="one_to_one",
)

for phenotype, csv_file in [
    ("functional_effect", func_effects_csv),
    ("latent_phenotype_effect", latent_effects_csv),
]:
    print(f"Writing {phenotype} to {csv_file}")
    (
        avg_func_effects.query("phenotype == @phenotype")[
            ["site", "wildtype", "mutant", "effect", "times_seen", "n_selections"]
        ].to_csv(csv_file, index=False, float_format="%.4g")
    )

## Make plots

Set up keyword arguments to [https://jbloomlab.github.io/polyclonal/polyclonal.plot.html#polyclonal.plot.lineplot_and_heatmap](https://jbloomlab.github.io/polyclonal/polyclonal.plot.html#polyclonal.plot.lineplot_and_heatmap) if they are not already specified:

In [ ]:
plot_kwargs = params["plot_kwargs"]

if "addtl_slider_stats" not in plot_kwargs:
    plot_kwargs["addtl_slider_stats"] = {}

if "times_seen" not in plot_kwargs["addtl_slider_stats"]:
    plot_kwargs["addtl_slider_stats"]["times_seen"] = 3

if "n_selections" not in plot_kwargs["addtl_slider_stats"]:
    plot_kwargs["addtl_slider_stats"]["n_selections"] = len(selections) // 2 + 1

if "site_zoom_bar_color_col" in plot_kwargs:
    if plot_kwargs["site_zoom_bar_color_col"] in avg_func_effects.columns:
        pass
    elif plot_kwargs["site_zoom_bar_color_col"] in site_numbering_map.columns:
        avg_func_effects = avg_func_effects.merge(
            site_numbering_map[["site", plot_kwargs["site_zoom_bar_color_col"]]],
            on="site",
            validate="many_to_one",
            how="left",
        )

if "addtl_tooltip_stats" not in plot_kwargs:
    plot_kwargs["addtl_tooltip_stats"] = []

if "sequential_site" not in avg_func_effects.columns:
    avg_func_effects = avg_func_effects.merge(
        site_numbering_map[["site", "sequential_site"]],
        on="site",
        validate="many_to_one",
        how="left",
    )
if any(avg_func_effects["site"] != avg_func_effects["sequential_site"]):
    if "sequential_site" not in plot_kwargs["addtl_tooltip_stats"]:
        plot_kwargs["addtl_tooltip_stats"].append("sequential_site")

if params["per_selection_tooltips"]:
    assert set(selections).issubset(avg_func_effects.columns)
    plot_kwargs["addtl_tooltip_stats"] += [
        s for s in selections if s not in plot_kwargs["addtl_tooltip_stats"]
    ]

if "alphabet" not in plot_kwargs:
    plot_kwargs["alphabet"] = [
        a
        for a in polyclonal.alphabets.biochem_order_aas(polyclonal.AAS_WITHSTOP_WITHGAP)
        if a in set(avg_func_effects["mutant"])
    ]

if "sites" not in plot_kwargs:
    plot_kwargs["sites"] = site_numbering_map.sort_values("sequential_site")[
        "site"
    ].tolist()

In [ ]:
for phenotype, plotfile in [
    ("functional_effect", functional_html),
    ("latent_phenotype_effect", latent_html),
]:
    print(f"\n\nPlotting {phenotype} and saving to {plotfile}")

    df = avg_func_effects.query("phenotype == @phenotype")

    chart = polyclonal.plot.lineplot_and_heatmap(
        data_df=df,
        stat_col="effect",
        category_col="phenotype",
        **plot_kwargs,
    )

    chart.save(plotfile)

    display(chart)